# Final Project

## problem

`-` 자신이 어떤 회사를 운영하거나 회사의 데이터 관리자라고 가정해보자 (예를 들어, 온라인 쇼핑몰을 운영하거나 데이터 분석 컨설팅 업체를 설립한다고 생각해볼 수 있겠다).  

`-` 회사의 데이터를 관리 및 활용하는 관점에서 데이터베이스를 설계하고, 필요한 데이터 분석 정보를 생성하는 Project를 수행하시오.

`-` 보고서에는 다음과 같은 사항이 포함되어야 합니다.

1. 필요한 데이터 항목, 데이터로부터 얻어야 되는 정보

2. 데이터베이스 구조 (테이블 수 및 테이블들간의 관계)

3. 데이터 업로드 과정 (데이터 import 과정, SQL 문장 등)

4. 데이터베이스에 업로드된 데이터 요약 (요약 통계량, 표/그래프 활용)

5. 필요한 정보를 얻기 위한 데이터 분석 (데이터 분석 기법 활용)

6. 결론 및 프로젝트 수행 소감

`-` Project 보고서는 1개의 파일로 만들어 첨부하시오. 

`-` R, Python code는 보고서에 부록으로 넣으시오.

`-` 보고서 발표 및 제출 마감일 : 2022. 6. 14(화) 22:00

## 할 것 정리

`-` 국민건강검진 데이터를 바탕으로 국민들의 건강을 check하고 건강 향상을 위한 정책이 필요한지를 판단하자

`-` 예컨대 흡연율이 증가하고 있다면 원인 파악은 주어진 데이터로는 모르겠지만 건강을 위해 금연 정책을 할 수는 있음

`-` 만약 흡연율이 감소하고 있다면 금연 정책에 들이는 노력을 다른 곳에 사용할지 고려할 수 있다

`-` 흡연율이 그대로라면 현재 시행하고 있는 금연 정책이 실효성이 있는지 파악해야 한다(흡연율이 왜 증가하는지 파악해야 함)

- 필요한 데이터 항목과 데이터로부터 얻어야 되는 정보

`-` 국민건강검진 데이터와 인구수와 병원수 데이터가 필요하다

`-` 데이터로부터 고혈압이나 당뇨 등의 비율을 알 수 있다

`-` 최근 3개년인 2018년~2020년 까지의 데이터는 일단 DB로 만들고 필요한 컬럼만 뽑아서 사용하자

`-` 예컨대 연도별 흡연율의 변화

`-` 2018~2020년 인구수 데이터 링크: https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_1B040A3

- 데이터베이스 구조 (테이블 수 및 테이블들간의 관계)

`-` 결측치가 많은 데이터끼리 모아 테이블을 만든다

`-` 나머지 컬럼에 대해서는 성격상 비슷한 것끼리 묶는다(내과, 안과 등)

`-` 테이블의 관계는 외래키를 통해 설명

- 나머지 part는 그냥 하면 된다

## 패키지 import 및 데이터 불러오기

In [1]:
import os
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
os.getcwd()

'C:\\Users\\Jaesu\\github_desktop\\statistical-database-project'

`-` 국민건강검진 데이터는 2018~2020년의 데이터를 하나의 파일로 묶겠다

`-` 결손치 유무, 제3대구치(사랑니) 이상, 치아마모증유무 변수는 2020년 데이터에는 존재하지 않는다

`-` 또한, 2018년과 2019년 데이터에서도 컬럼만 존재하지 미시행하였기 때문에 drop 하겠다

`-` 가입자 일련번호를 1~3000000으로 set 해주고 데이터 공개일자 변수는 기준년도 변수가 있으므로 drop 하겠다

In [209]:
def remove_space(df: pd.DataFrame) -> pd.DataFrame:
    """컬럼명에 포함된 공백을 제거한 데이터프레임을 반환하는 함수"""
    df = df.rename(columns = dict(zip(df.columns, [column.replace(' ', '') for column in df.columns]))) 
    return df

In [210]:
def get_NHIS_dataframe() -> pd.DataFrame:
    """
    NHIS_2018 ~ NHIS_2020 csv 파일을 불러온 후 간단한 전처리를 수행하고 merge 하여 반환하는 함수
    
    *--- 전처리 ---*
    1. NHIS_2019 데이터에 중복행이 있어 이를 제거
    2. csv 파일의 컬럼명을 서로 동일하게 해주기 위해 각 csv 파일의 컬럼명에 있는 공백을 제거
    
    *--- Data Path ---*
    .(current directory)
    |--- Data
        |--- NHIS_OPEN_GJ_2018.csv
        |--- NHIS_OPEN_GJ_2019.csv
        |--- NHIS_OPEN_GJ_2020.csv
    """
    df = pd.concat([
                    remove_space(pd.read_csv('./Data/NHIS_OPEN_GJ_2018.csv', encoding='euc=kr')),

                    remove_space(pd.read_csv('./Data/NHIS_OPEN_GJ_2019.csv', encoding='euc=kr', on_bad_lines='skip').
                                    drop_duplicates(). ## 중복행 제거
                                    assign(**{'가입자 일련번호': lambda df: df['가입자 일련번호'] + 1000000})), ## 2019년도는 일련번호 1000001부터 시작
                    
                    remove_space(pd.read_csv('./Data/NHIS_OPEN_GJ_2020.csv', encoding='euc=kr').
                                    assign(**{'가입자 일련번호': lambda df: df['가입자 일련번호'] + 2000000})) ## 2020년도는 일련번호 2000001부터 시작
                   ]).reset_index().drop('index', axis=1)
    return df

In [211]:
df = get_NHIS_dataframe() ## 행이 3백만개라 시간이 조금 걸린다

- 데이터프레임의 간단한 정보

In [214]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 34 columns):
 #   Column        Dtype  
---  ------        -----  
 0   기준년도          int64  
 1   가입자일련번호       int64  
 2   성별코드          int64  
 3   연령대코드(5세단위)   int64  
 4   시도코드          int64  
 5   신장(5Cm단위)     int64  
 6   체중(5Kg단위)     int64  
 7   허리둘레          float64
 8   시력(좌)         float64
 9   시력(우)         float64
 10  청력(좌)         float64
 11  청력(우)         float64
 12  수축기혈압         float64
 13  이완기혈압         float64
 14  식전혈당(공복혈당)    float64
 15  총콜레스테롤        float64
 16  트리글리세라이드      float64
 17  HDL콜레스테롤      float64
 18  LDL콜레스테롤      float64
 19  혈색소           float64
 20  요단백           float64
 21  혈청크레아티닌       float64
 22  (혈청지오티)AST    float64
 23  (혈청지오티)ALT    float64
 24  감마지티피         float64
 25  흡연상태          float64
 26  음주여부          object 
 27  구강검진수검여부      object 
 28  치아우식증유무       float64
 29  결손치유무         object 
 30  치아마모증유무       obje

- 결측치

In [216]:
df.isnull().sum()

기준년도                  0
가입자일련번호               0
성별코드                  0
연령대코드(5세단위)           0
시도코드                  0
신장(5Cm단위)             0
체중(5Kg단위)             0
허리둘레                925
시력(좌)               705
시력(우)               706
청력(좌)               620
청력(우)               625
수축기혈압             18686
이완기혈압             18689
식전혈당(공복혈당)        19040
총콜레스테롤          1931389
트리글리세라이드        1931386
HDL콜레스테롤        1931398
LDL콜레스테롤        1952503
혈색소               19057
요단백               33017
혈청크레아티닌           19034
(혈청지오티)AST        19029
(혈청지오티)ALT        19031
감마지티피             19042
흡연상태                743
음주여부             710221
구강검진수검여부              0
치아우식증유무         1870945
결손치유무           2000000
치아마모증유무         2000000
제3대구치(사랑니)이상    2000000
치석              1893308
데이터공개일자               0
dtype: int64

`-` 위에서 언급했듯이 결손치 유무, 제3대구치(사랑니) 이상, 치아마모증유무 변수는 drop 하겠다

In [5]:
def drop_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    결손치유무, 제3대구치(사랑니)이상, 치아마모증유무 변수를 drop한 데이터프레임을 반환하는 함수(원본이 변한다)
    """
    df = df.drop(['결손치유무', '제3대구치(사랑니)이상', '치아마모증유무'], axis=1)
    return df

In [8]:
# df = drop_columns(df) 

`-` 전처리가 끝난 csv 파일은 db로 올려야 한다!!!(물론 테이블을 나누고)

In [215]:
df2 = pd.read_csv('./Data/행정구역_시군구별_성별_인구수.csv', encoding='euc-kr')

In [31]:
df3 = pd.read_csv('./Data/시군구별_표시과목별_의원_현황.csv', encoding='euc-kr')

In [30]:
df2 ## 데이터 확인

,행정구역(시군구)별,2018,2018.1,2018.2,2019,2019.1,2019.2,2020,2020.1,2020.2
0,행정구역(시군구)별,총인구수 (명),남자인구수 (명),여자인구수 (명),총인구수 (명),남자인구수 (명),여자인구수 (명),총인구수 (명),남자인구수 (명),여자인구수 (명)
1,전국,51826059,25866129,25959930,51849861,25864816,25985045,51829023,25841029,25987994
2,서울특별시,9765623,4773899,4991724,9729107,4744059,4985048,9668465,4701723,4966742
3,부산광역시,3441453,1692421,1749032,3413841,1675417,1738424,3391946,1661000,1730946
4,대구광역시,2461769,1219342,1242427,2438031,1205286,1232745,2418346,1193109,1225237
5,인천광역시,2954642,1481844,1472798,2957026,1482249,1474777,2942828,1473943,1468885
6,광주광역시,1459336,722581,736755,1456468,720740,735728,1450062,717348,732714
7,대전광역시,1489936,744338,745598,1474870,736607,738263,1463882,730699,733183
8,울산광역시,1155623,593819,561804,1148019,589712,558307,1136017,583708,552309
9,세종특별자치시,314126,156831,157295,340575,169845,170730,355831,177568,178263


In [34]:
df3 ## 데이터 확인

,시군구별(1),2022.1/4,2022.1/4.1,2022.1/4.2,2022.1/4.3,2022.1/4.4,2022.1/4.5,2022.1/4.6,2022.1/4.7,2022.1/4.8,...,2022.1/4.19,2022.1/4.20,2022.1/4.21,2022.1/4.22,2022.1/4.23,2022.1/4.24,2022.1/4.25,2022.1/4.26,2022.1/4.27,2022.1/4.28
0,시군구별(1),계,내과,신경과,정신건강의학과,외과,정형외과,신경외과,흉부외과,성형외과,...,진단검사의학과,결핵과,재활의학과,핵의학과,가정의학과,응급의학과,직업환경의학과,예방의학과,전문과목미표시전문의,일반의
1,전체,34216,5148,264,1442,1023,2378,560,57,1110,...,11,1,477,1,844,0,1,1,6183,3088
2,서울,9264,1188,48,500,220,606,105,10,600,...,4,0,165,0,204,0,0,0,1718,766
3,부산,2493,414,20,128,62,163,39,8,100,...,0,0,26,1,40,0,0,0,399,235
4,대구,1903,354,32,73,76,143,47,3,65,...,1,0,16,0,32,0,0,0,252,99
5,인천,1664,254,8,53,49,139,28,1,34,...,0,0,23,0,31,0,0,0,321,156
6,광주,988,166,12,43,31,43,17,0,28,...,0,0,17,0,48,0,0,0,164,83
7,대전,1122,184,17,62,31,79,11,4,33,...,2,0,35,0,30,0,0,0,168,108
8,울산,621,94,3,17,13,46,8,5,12,...,0,0,6,0,17,0,0,0,118,60
9,세종,203,30,2,9,7,14,4,0,4,...,0,0,2,0,6,0,0,0,28,16
